In [1]:
using Langevin
import Serialization
holstein = Serialization.deserialize("holstein_6x6_cond106.dat")
;

┌ Info: Recompiling stale cache file /Users/kbarros/.julia/compiled/v1.1/Langevin/nUpf1.ji for Langevin [d56e79bc-9d16-11e9-1337-85b17f59915c]
└ @ Base loading.jl:1184


In [2]:
using LinearAlgebra
using IterativeSolvers
using SparseArrays
using Random

M = Array(Langevin.HolsteinModels.construct_M(holstein))
cond(M)

105.99637038875237

# Accuracy measurements

In [3]:
Random.seed!(0)
b = randn(size(holstein)[1])

x_exact = M \ b
x_approx = fill!(similar(b), 0)
;

In [4]:
rhs = M' * b
fill!(x_approx, 0)
x1 = IterativeSolvers.cg!(x_approx, holstein, rhs, tol=1e-4, log=true, maxiter=1000)
println("CG iters ", x1[2].iters)
println("True error ", norm(x_approx - x_exact) / norm(x_exact))

CG iters 286
True error 0.005060902377033015


In [5]:
using Langevin.Preconditioners: MatrixMOp, BlockPreconditioner, FourierPreconditioner
M_op = MatrixMOp(holstein)
P_op = BlockPreconditioner(holstein, subtol=1e-2)
PF_op = FourierPreconditioner(holstein)
;

In [6]:
fill!(x_approx, 0)
x2 = IterativeSolvers.gmres!(x_approx, M_op, b, Pr=P_op, tol=2e-3, restart=10, log=true, maxiter=100, initially_zero=true, verbose=true)

println("GMRES with block preconditioner")
println("True error ", norm(x_approx - x_exact) / norm(x_exact))

=== gmres ===
rest	iter	resnorm
  1	  1	8.34e+00
  1	  2	2.88e+00
  1	  3	1.29e+00
  1	  4	4.81e-01
  1	  5	2.05e-01
  1	  6	7.68e-02

GMRES with block preconditioner
True error 0.004140905302401464


In [7]:
fill!(x_approx, 0)
x2 = IterativeSolvers.gmres!(x_approx, M_op, b, Pr=PF_op, tol=2e-3, restart=10, log=true, maxiter=100, initially_zero=true, verbose=true)

println("GMRES with Fourier preconditioner")
println("True error ", norm(x_approx - x_exact) / norm(x_exact))

=== gmres ===
rest	iter	resnorm
  1	  1	9.68e+00
  1	  2	4.28e+00
  1	  3	2.19e+00
  1	  4	1.24e+00
  1	  5	7.24e-01
  1	  6	4.14e-01
  1	  7	2.14e-01
  1	  8	9.92e-02

GMRES with Fourier preconditioner
True error 0.005586265309813574


In [8]:
# BiCGStab uses internal randomness. :-(
# Set seed to get reproducible results.
Random.seed!(0)

fill!(x_approx, 0)
x2 = IterativeSolvers.bicgstabl!(x_approx, M_op, b, 1, Pl=PF_op, tol=1e-2, log=true, max_mv_products=100, initial_zero=true, verbose=true)

println("BiCGStab(1) with Fourier preconditioner")
println("True error ", norm(x_approx - x_exact) / norm(x_exact))

  1	6.59e+01
  2	8.09e+00
  3	6.59e+01
  4	3.68e+00
  5	3.19e-01

BiCGStab(1) with Fourier preconditioner
True error 0.0033509204835340724


# Benchmarking

In [9]:
using BenchmarkTools

println("CG time...")
@btime begin
    fill!(x_approx, 0)
    IterativeSolvers.cg!(x_approx, holstein, rhs, tol=1e-4, log=true, maxiter=1000)
end
;

CG time...
  2.567 ms (880 allocations: 90.08 KiB)


In [10]:
println("GMRES/Block time...")
@btime begin
    fill!(x_approx, 0)
    IterativeSolvers.gmres!(x_approx, M_op, b, Pr=P_op, tol=2e-3, restart=10, maxiter=100, initially_zero=true)
end
;

GMRES/Block time...
  2.062 ms (18407 allocations: 1.37 MiB)


In [11]:
println("GMRES/Fourier time...")
@btime begin
    fill!(x_approx, 0)
    IterativeSolvers.gmres!(x_approx, M_op, b, Pr=PF_op, tol=2e-3, restart=10, maxiter=100, initially_zero=true)
end
;

GMRES/Fourier time...
  953.505 μs (253 allocations: 285.20 KiB)


In [15]:
println("GMRES/Fourier time, prealloc...")

gmres = IterativeSolvers.gmres_iterable!(x_approx, M_op, b, Pr=PF_op, tol=2e-3, restart=10, maxiter=100, initially_zero=true)

@btime begin
    fill!(x_approx, 0)
    Langevin.Preconditioners.reset_gmres_iterable!(gmres, x_approx, M_op, b, tol=2e-3, initially_zero=true)
    Langevin.Preconditioners.run_gmres_iterable!(gmres)
end
;

GMRES/Fourier time, prealloc...
  850.559 μs (236 allocations: 12.94 KiB)


In [16]:
println("BiCGStab(1)/Fourier time...")

Random.seed!(0)

@btime begin
    fill!(x_approx, 0)
    x2 = IterativeSolvers.bicgstabl!(x_approx, M_op, b, 1, Pl=PF_op, tol=1e-2, max_mv_products=100, initial_zero=true)
end
;

BiCGStab(1)/Fourier time...
  833.554 μs (259 allocations: 486.70 KiB)


# Profiling

For some reason, it appears that the resulting flamegraphs are not entirely trustworthy.

In [6]:
using Profile
Profile.clear()  # in case we have any previous profiling data

@profile begin
    for i = 1:100
        fill!(x_approx, 0)
        IterativeSolvers.bicgstabl!(x_approx, M_op, b, 1, Pl=PF_op, tol=1e-2, max_mv_products=100, initial_zero=true)
    end
end

In [ ]:
Serialization.serialize("serialized_profile.dat", Profile.retrieve())

In [19]:
Profile.print()

In [ ]:
r = Serialization.deserialize("serialized_profile.dat")
using ProfileView
ProfileView.view(r[1], lidict=r[2])
ProfileView.svgwrite("flamegraph_gmres.svg",r[1],r[2])